# An example workflow for generating hypotheses with ABCD data

## Installing dependencies
One way to do this is with a virtual python environment installed so that it is accessible to Jupyter lab.  This needs to be set up only once.
```bash
python -m venv ~/abcd311
source ~/abcd311/bin/activate
pip install ipykernel
python -m ipykernel install --user --name abcd311
pip install -r requirements.txt
```
Then, once Jupyter is open with this lab notebook, "Change Kernel..." to be "abcd311".

In [ ]:
# Import from Python packages
from dipy.io.image import load_nifti, save_nifti
import csv
import functools
import math
import numpy as np
import os
import pandas as pd
import random
import re

In [ ]:
# Set global parameters to match your environment
gor_image_directory = "/data2/ABCD/gor-images"
coregistered_images_directory = os.path.join(gor_image_directory, "coregistered-images")
tabular_data_directory = "/data2/ABCD/abcd-5.0-tabular-data-extracted"
core_directory = os.path.join(tabular_data_directory, "core")

In [ ]:
# The locations of some useful csv data columns

# These files live in `core_directory`

independent_vars = [
    [
        "abcd-general/abcd_y_lt.csv",
        [
            "site_id_l",  # Site ID at each event
            # TODO: We are including participants with no siblings in study at the expense of losing family ID.  Do this better.
            # "rel_family_id",  # Participants belonging to the same family share a family ID.  They will differ between data releases
            "interview_age",  # Participant's age in month at start of the event
        ],
    ],
    [
        "abcd-general/abcd_p_demo.csv",
        [
            "demo_gender_id_v2",  # 1=Male; 2=Female; 3=Trans male; 4=Trans female; 5=Gender queer; 6=Different; 777=Decline to answer; 999=Don't know
            # "demo_gender_id_v2_l",  # same?
        ],
    ],
    [
        "gender-identity-sexual-health/gish_p_gi.csv",
        [
            "demo_gender_id_v2",  # 1=Male; 2=Female; 3=Trans male; 4=Trans female; 5=Gender queer; 6=Different; 777=Decline to answer; 999=Don't know
            # "demo_gender_id_v2_l",  # same?
        ],
    ],
    [
        "physical-health/ph_y_bld.csv",
        [
            # "biospec_blood_baso_percent",  # BASO %
            # "biospec_blood_baso_abs",  # BASO ABS
            # "biospec_blood_eos_percent",  # EOS %
            # "biospec_blood_eos_abs",  # EOS ABS
            # "biospec_blood_hemoglobin",  # Hemoglobin
            # "biospec_blood_mcv",  # MCV
            # "biospec_blood_plt_count",  # PLT Count
            # "biospec_blood_wbc_count",  # WBC Count
            # "biospec_blood_ferritin",  # Ferritin
            # "biospec_blood_hemoglobin_a1",  # hemoglobin_a1
            # "biospec_blood_imm_gran_per",  # Immature Gran %
        ],
    ],
]

# We'll do KSADS variables once we've computed `interesting_ksads` below

In [ ]:
# Functions for handling image voxel data


def get_list_of_image_files(directory):
    response = [
        os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".nii.gz")
    ]
    return response


def parse_image_filenames(list_of_image_files):
    """
    Returns a pandas DataFrame.
    The first column is the filename.  Additional columns indicate how the filename was parsed.
    For example, run as:
        df = parse_image_filenames(get_list_of_image_files(coregistered_images_directory))
    """
    filename_pattern = r"gorinput([0-9]+)-modality([0-9]+)-sub-([A-Za-z0-9]+)_ses-([A-Za-z0-9]+)_run-([A-Za-z0-9]+)_([A-Za-z0-9]+)_([A-Za-z0-9]+)-([A-Za-z0-9]+).nii.gz"
    filename_keys = [
        "filename",
        "gorinput",
        "modality",
        "src_subject_id",
        "eventname",
        "run",
        "image_type",
        "image_subtype",
        "processing",
    ]

    response = pd.DataFrame(
        [
            [filename, *list(re.match(filename_pattern, os.path.basename(filename)).groups())]
            for filename in list_of_image_files
        ],
        columns=filename_keys,
    )
    # Fix parsing of src_subject_id
    response["src_subject_id"] = [
        re.sub(r"^NDAR", "NDAR_", subject) for subject in response["src_subject_id"]
    ]
    # Fix parsing of eventname
    eventname_conversion = {
        "baselineYear1Arm1": "baseline_year_1_arm_1",
        "1YearFollowUpYArm1": "1_year_follow_up_y_arm_1",
        "2YearFollowUpYArm1": "2_year_follow_up_y_arm_1",
        "3YearFollowUpYArm1": "3_year_follow_up_y_arm_1",
        "4YearFollowUpYArm1": "4_year_follow_up_y_arm_1",
    }
    response["eventname"] = [eventname_conversion[event] for event in response["eventname"]]
    return response


def get_data_from_image_files(list_of_files):
    """
    get_data_from_image_files returns a list of tuples of 4 values:
    <class 'str'>: full file name
    <class 'numpy.ndarray'>: image data as a numpy array
    <class 'numpy.ndarray'>: some other numpy array (transform?)
    <class 'nibabel.nifti1.Nifti1Image'>: an object that can be modified and written to file as a .nii.gz file
    """
    response = [(file,) + load_nifti(file, return_img=True) for file in list_of_files]
    return response

In [ ]:
# Functions for reading and selecting data from csv files


def csv_file_to_dataframe(filename):
    return pd.read_csv(filename)


# Select rows from data frame, to handle a common case where the direct pandas interface would be complicated
def select_rows_of_dataframe(df, query_dict):
    # Each key of query_dict is a column header of the df dataframe.
    # Each value of query_dict is a list of allowed values.
    # A row will be selected only if each of these columns has one of the allowed keys
    assert all(key in df.columns for key in query_dict.keys())
    # Old code: each of query_dict.values() is just a single value, not a list of values:
    # rows = df[
    #     functools.reduce(lambda x, y: x & y, [df[key] = value for key, value in query_dict.items()])
    # ]
    rows = df[
        functools.reduce(
            lambda x, y: x & y,
            [
                functools.reduce(lambda x, y: x | y, [df[key] == value for value in values])
                for key, values in query_dict.items()
            ],
        )
    ]
    return rows

In [ ]:
# Functions for computing summary statistics for KSADS csv data


def data_frame_value_counts(df):
    # Returns a dict:
    #     Each key is a column name
    #     Each value is a dict:
    #         Each key of this is a value that occurs in the column.
    #         The corresponding value is the number of occurrences.
    for column in df.columns:
        if bool(re.match("ksads_\d", column)):
            df.loc[df[column] == 555, column] = np.nan
            df.loc[df[column] == 888, column] = 0
    return {
        column: dict(df[column].value_counts(dropna=False).astype(int)) for column in df.columns
    }


def entropy_of_column_counts(column_counts):
    assert all(value >= 0 for value in column_counts.values())
    total_count = sum(column_counts.values())
    entropy = sum(
        [
            count / total_count * math.log2(total_count / count)
            for count in column_counts.values()
            if count > 0
        ]
    )
    return entropy


def ksads_keys_only(all_columns):
    return {key: value for key, value in all_columns.items() if bool(re.match("ksads_\d", key))}


def entropy_of_all_columns(all_columns):
    return {key: entropy_of_column_counts(value) for key, value in all_columns.items()}


def find_interesting_entropies(file_mh_y_ksads_ss):
    # Find some KSADS data columns with high entropy.
    df_mh_y_ksads_ss = csv_file_to_dataframe(file_mh_y_ksads_ss)
    print("Read done")

    counts_for_each_column_mh_y_ksads_ss = ksads_keys_only(
        data_frame_value_counts(df_mh_y_ksads_ss)
    )
    print("Column counting done")

    entropies = entropy_of_all_columns(counts_for_each_column_mh_y_ksads_ss)
    sorted_entropies = dict(sorted(entropies.items(), key=lambda item: item[1], reverse=True))
    sorted_entropies = {
        key: (value, counts_for_each_column_mh_y_ksads_ss[key])
        for key, value in sorted_entropies.items()
        # if bool(re.match("ksads_\d", key))
    }
    print("Entropy calculation done")
    return sorted_entropies

In [ ]:
# Find interesting KSADS data

file_mh_y_ksads_ss = "mental-health/mh_y_ksads_ss.csv"
if True:
    # User requests that we re-compute `interesting_ksads`
    if False:
        # User requests that we re-compute `sorted_entropies`
        try:
            del sorted_entropies
        except NameError:
            pass
    try:
        sorted_entropies
        print("Using cached value for sorted_entropies")
    except NameError:
        full_path = os.path.join(core_directory, file_mh_y_ksads_ss)
        # find_interesting_entropies is slow
        sorted_entropies = find_interesting_entropies(full_path)
    number_wanted = 20
    print(f"Distribution details: {list(sorted_entropies.items())[:number_wanted]}")
    interesting_ksads = list(sorted_entropies.keys())[:number_wanted]
    print(f"{interesting_ksads = }")
else:
    # With "555" and "888" both going to "", the interesting_ksads computation gives:
    interesting_ksads = [
        "ksads_22_142_t",  # Symptom - Insomnia, Past
        "ksads_22_970_t",  # Diagnosis - SLEEP PROBLEMS, Past
        "ksads_2_11_t",  # Symptom - Explosive Irritability, Past
        "ksads_1_2_t",  # Symptom - Depressed Mood, Past
        "ksads_2_13_t",  # Symptom - Decreased Need for Sleep, Past
        "ksads_1_6_t",  # Symptom - Anhedonia, Past
        "ksads_22_141_t",  # Symptom - Insomnia, Present
        "ksads_22_969_t",  # Diagnosis - SLEEP PROBLEMS, Present
        "ksads_2_8_t",  # Symptom - Elevated Mood, Past
        "ksads_10_46_t",  # Symptom - Excessive worries more days than not Past
        "ksads_1_4_t",  # Symptom - Irritability, Past
        "ksads_2_10_t",  # Symptom - ExplosiveIrritability, PresentNext
        "ksads_8_31_t",  # Symptom - Fear of Social Situations, Past
        "ksads_23_146_t",  # Symptom - Wishes/Better off dead, Past
        "ksads_23_957_t",  # Diagnosis - SuicidalideationPassivePast
        "ksads_1_5_t",  # Symptom - Anhedonia, Present
        "ksads_2_839_t",  # Diagnosis - Unspecified Bipolar and Related Disorder, PAST (F31.9)
        "ksads_2_833_t",  # Diagnosis - Bipolar I Disorder, most recent past episode manic (F31.1x)
        "ksads_1_3_t",  # Symptom - Irritability, Present
        "ksads_1_842_t",  # Diagnosis - Major Depressive Disorder, Past (F32.9)
    ]
    # With "555" going to "" and "888" going to "0", the interesting_ksads computation gives:
    interesting_ksads = [
        "ksads_1_187_t",  # Symptom - No two month symptom-free interval, Present
        "ksads_1_188_t",  # Symptom - No two month symptom-free interval, Past
        "ksads_22_142_t",  # Symptom - Insomnia, Past
        "ksads_22_970_t",  # Diagnosis - SLEEP PROBLEMS, Past
        "ksads_2_11_t",  # Symptom - Explosive Irritability, Past
        "ksads_2_222_t",  # Symptom - Lasting at least 4 days, Past
        "ksads_1_184_t",  # Symptom - Impairment in functioning due to depression, Past
        "ksads_1_2_t",  # Symptom - Depressed Mood, Past
        "ksads_2_13_t",  # Symptom - Decreased Need for Sleep, Past
        "ksads_1_6_t",  # Symptom - Anhedonia, Past
        "ksads_1_160_t",  # Symptom - Fatigue, Past
        "ksads_1_162_t",  # Symptom - Concentration Disturbance, Past
        "ksads_2_220_t",  # Symptom - Lasting at least one week, Past
        "ksads_1_156_t",  # Symptom - Insomnia when depressed, Past
        "ksads_1_174_t",  # Symptom - Psychomotor Agitation in Depressive Disorder, Past
        "ksads_2_216_t",  # Symptom - Impairment in functioning due to bipolar, Past
        "ksads_2_208_t",  # Symptom - Psychomotor Agitation in Bipolar Disorder, Past
        "ksads_2_206_t",  # Symptom - Increased Energy, Past
        "ksads_22_141_t",  # Symptom - Insomnia, Present
        "ksads_22_969_t",  # Diagnosis - SLEEP PROBLEMS, Present
    ]

ksads_vars = [[file_mh_y_ksads_ss, interesting_ksads]]

In [ ]:
# Find all images for which we have tabular data

join_keys = ["src_subject_id", "eventname"]


def get_table_drop_nulls(tablename, list_of_keys):
    df = csv_file_to_dataframe(os.path.join(core_directory, tablename))[list_of_keys]
    df.replace("", pd.NA, inplace=True)
    df.dropna(inplace=True)
    print(df.head())
    return df


def merge_dataframes_for_keys(independent_vars):
    df_generator = (
        get_table_drop_nulls(tablename, [*join_keys, *list_of_keys])
        for tablename, list_of_keys in independent_vars
        if list_of_keys
    )
    df_all_keys = next(df_generator)
    for df_next in df_generator:
        df_all_keys = pd.merge(
            df_all_keys, df_next, on=join_keys, how="inner", validate="one_to_one"
        )
    return df_all_keys


def merge_tabular_information(independent_vars, coregistered_images_directory):
    df_all_keys = merge_dataframes_for_keys(independent_vars)

    list_of_image_files = get_list_of_image_files(coregistered_images_directory)
    df_image_information = parse_image_filenames(list_of_image_files)
    df_all_images = pd.merge(
        df_all_keys,
        df_image_information[[*join_keys, "image_subtype"]],
        on=join_keys,
        how="inner",
        validate="one_to_many",
    )
    return df_all_images


tabular_information = merge_tabular_information(independent_vars, coregistered_images_directory)
print(f"{len(tabular_information) = }")